In [8]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import joblib

In [2]:
df = pd.read_csv("ObesityDataSet.csv")
print(df.shape)
df.head()

(2111, 17)


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
cat_features = df.select_dtypes(include = ['object'])
cat_features.columns

Index(['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE',
       'SCC', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [4]:
for i in cat_features:
    print(f"{i} : {df[i].unique()}")

Gender : ['Female' 'Male']
family_history_with_overweight : ['yes' 'no']
FAVC : ['no' 'yes']
CAEC : ['Sometimes' 'Frequently' 'Always' 'no']
SMOKE : ['no' 'yes']
SCC : ['no' 'yes']
CALC : ['no' 'Sometimes' 'Frequently' 'Always']
MTRANS : ['Public_Transportation' 'Walking' 'Automobile' 'Motorbike' 'Bike']
NObeyesdad : ['Normal_Weight' 'Overweight_Level_I' 'Overweight_Level_II'
 'Obesity_Type_I' 'Insufficient_Weight' 'Obesity_Type_II'
 'Obesity_Type_III']


In [6]:
le = LabelEncoder()
for i in cat_features:
    df[i] = le.fit_transform(df[i])

In [7]:
for i in cat_features:
    print(f"{i} : {df[i].unique()}")

Gender : [0 1]
family_history_with_overweight : [1 0]
FAVC : [0 1]
CAEC : [2 1 0 3]
SMOKE : [0 1]
SCC : [0 1]
CALC : [3 2 1 0]
MTRANS : [3 4 0 2 1]
NObeyesdad : [1 5 6 2 0 3 4]


In [10]:
X = df.drop('NObeyesdad', axis=1) 
y = df['NObeyesdad']

In [11]:
cv_scores_train = []
cv_scores_test = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model_ANN = Sequential()
    model_ANN.add(Dense(64, kernel_initializer = 'uniform', activation='relu', input_shape=(X_train.shape[1],)))
    model_ANN.add(Dense(64, kernel_initializer = 'uniform', activation='relu'))
    model_ANN.add(Dense(7, kernel_initializer = 'uniform', activation='softmax'))

    model_ANN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    # y_train = to_categorical(y_train)
    model_ANN.fit(X_train_scaled, y_train, epochs=40, batch_size=45, verbose=0)

    train_score = model_ANN.evaluate(X_train_scaled, y_train, verbose=0)[1]  # Accuracy is at index 1
    test_score = model_ANN.evaluate(X_test_scaled, y_test, verbose=0)[1]  # Accuracy is at index 1

    cv_scores_train.append(train_score)
    cv_scores_test.append(test_score)

mean_cv_score_train = np.mean(cv_scores_train)
print(f"Mean Cross-Validation Training Accuracy: {mean_cv_score_train}")
mean_cv_score_test = np.mean(cv_scores_test)
print(f"Mean Cross-Validation Testing Accuracy: {mean_cv_score_test}")

/home/surajprajapati/Documents/Hackthonpring24/ObesityForecast.com/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/surajprajapati/Documents/Hackthonpring24/ObesityForecast.com/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/surajprajapati/Documents/Hackthonpring24/ObesityForecast.com/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. Wh

Mean Cross-Validation Training Accuracy: 0.9990521430969238
Mean Cross-Validation Testing Accuracy: 0.9569739937782288
